# An RNN Sequence tagger for annotation of ECGs

This example shows how to set-up a sequence tagger and how to integrate self-written
recurrent units with the NNHelferlein ecosystem.

The goal is, to annotate ECG recordings with the heart beats. 
As training data, a reduced set of 18 long-term ECG recordings have been prepared form the Physionet-Database
(MIT-BIH Normal Sinus Rhythm Database, doi: https://doi.org/10.13026/C2NK5R).

The ECGs are provided with a sampling rate of 128 points per sec and short windows are sampled 
from the data for training.
To include about two heart beats in each window, sequences of 200 time steps are generated. 
To be able to handle these long sequences, we use Peephole-LSTM units which are said 
to be a good choice for long sequences with rare peaks.

In [1]:
using Knet: param, param0, sigm, Adam
using NNHelferlein
using CSV, DataFrames, Statistics
using Plots
using ImageFiltering

In [2]:
WIN = 200             # window size in time steps (rate = 128 per sec)
SMOOTH = 11           # window size of kernel smoothing
UNITS = 8             # number of recurrent units
MB = 512              # minibatch size
TRN = 1:15            # ECGs for training
TST = 16:18           # ECGs for test
NAME="Beat-finder-$UNITS"    # name for TensorBoard log

"Beat-finder-8"

## Dataset:
The 18 data files are provided as part of the NNhelferlein distribution as csv-files 
16265, 16272, 16273, 16420, 16483, 16539, 16773, 16786, 16795,
17052, 17453, 18177, 18184, 19088, 19090, 19093, 19140, 19830.
The files comprise the 3 columns `normal heart beat 0/1`, `channel 1` 
and `channel 2` of the ECG.
Each file includes apount 50 000 heart beats and 15 of the files are used for training and validation
and 3 for testing.

To make the classification task a little bit less unbalanced the *beat* column (that displays
many zeros and only single ones to mark the heart beats) is replaced by a 
*smooth* column by applying a gaussian filter to the beats. The shape of this new 
teaching input (or training target) is shown as green line in the first plot 
(in the later plots, the green line shows the prediction!).

The importer function reads the training files and samples randomly windows of length 200 from the 
ECGs. The number of windows is adjusted, so that the total number of steps in all windows equals the
total number of time steps in the source file (in analogy to a 0.632 bootstrap).

In [3]:
function readEcg(record)
    fname = joinpath(NNHelferlein.DATA_DIR, "MIT-normal_sinus_rhythm", record)
    x = CSV.File(fname, header=["beat", "cha1", "cha2"], 
                   types=[UInt8, Float32, Float32]) |> DataFrame
    x.time = collect(1:nrow(x))./128 
    
    # smooth beats:
    #
    σ = 2.0; l = SMOOTH
    w = ImageFiltering.KernelFactors.gaussian(σ, l)
    sc = maximum(w)
    x.smooth = Float32.(imfilter(x.beat, w, Fill(0,w)) ./ sc)
    return x
end

readEcg (generic function with 1 method)

In [4]:
files = readdir(joinpath(NNHelferlein.DATA_DIR, "MIT-normal_sinus_rhythm"))
nsr = [readEcg(nam) for nam in files]
nsr[2][1:3,:]

,beat,cha1,cha2,time,smooth
,UInt8,Float32,Float32,Float64,Float32
1,0,-0.032,-0.0541,0.0078125,0.0
2,0,-0.032,0.0582,0.015625,0.0
3,0,-0.032,0.0582,0.0234375,0.0


#### Plotting the ECGs:

In [5]:
function plotWindow(t, ch1, ch2, beats, preds=nothing)
    
    plt = plot(t, ch1, ylim=(-3.1,3), color=:black, 
               yticks=false, legends=false)
    plt = plot(plt, t, ch2.+2, color=:blue)
    plt = plot(plt, t, beats.-2, color=:red)
    if !isnothing(preds)
        preds = Array(preds)
        plt = plot(plt, t, preds.-3, color=:green)
    end
    display(plt)
    return plt
end

function plotECG(x, start; win=1000)
    range = start:start+win
    if "smooth" in names(x)
        p = x.smooth[range]
    else
        p = nothing
    end
    plotWindow(x.time[range], x.cha1[range], x.cha2[range], x.beat[range], p)
end

plotECG (generic function with 1 method)

Plot illustrating the training data:
+ *black, blue:* channel 1 and 2 of the ecg, scaled to a range
  from-1 to 1
+ *red:* annotated heart beats; only one time step marks the exact 
  position of the beats
+ *green:* smoothed beats as teaching input.

In [ ]:
plotECG(nsr[1], 1000000);

<img src="assets/62-data.png">

#### Window sampling:

Minibaches are constructed from randomly sampled windows from the training data:

In [7]:
function random_window_sample(df...; win=200, n=nothing)
    x = []; y = []
    
    num_df = length(df)
    total = reduce(+, nrow.(df))
    if isnothing(n)
        n = cld(total, win)
    end
    println("Sampling $n windows from $num_df ECGs")

    for i in 1:n
        dfs = df[rand(1:num_df)]
        rows = nrow(dfs)
    
        wstart = rand(1:rows-win+1)
        wrange = wstart:wstart+win-1
        push!(x, permutedims(Array(dfs[wrange,[:cha1, :cha2]]), (2,1)))
        push!(y, Array(dfs[wrange,:smooth]))
    end
    return (x, y)
end

random_window_sample (generic function with 1 method)

In [8]:
x,y = random_window_sample(nsr[TRN]..., win=WIN);

Sampling 369801 windows from 15 ECGs


In [9]:
dtrn = sequence_minibatch(x,y, MB, seq2seq=false);

In [10]:
x,y = first(dtrn);

## Peephole LSTM

The peephole LSTM unis are defined as a subtype of `RecurrentUnit` with a signature for
forward calculation of one minibatch.
Design follows the original paper of Gers & Schmidhuber 
*(Learning Precise Timing with LSTM Recurrent Networks, 
Journal of Machine Learning Research 3 (2002) 115-143):*

In the formulas, $x$ denotes the layer input and 
$h_{t-1}$ the recurrent connections; i.e. the hidden state of
the last step.   
$g$ denotes any of the gate openings $g_i, g_o, g_f$.

#### Input, output and forget gates:
In addtion to the vanilla LSTM gate function, the current cell state $c_{t-1}$,
multiplied with a trainable weight $c_g$, is taken into account here
(for a single unit, $c_g$ and $c$ are scalars; to calculate gates for 
a complete layer, the respective vectors has the length of the number of units
and must be multiplicated element-wise, denoted as $\odot$):

$$ g_{g\in(i,o,f)} = \sigma(W_g \cdot [h_{t-1}, x_t] + b_g + c_{g} \odot c_{t-1})$$

#### Cell state:
The cell state is updated the same way as in normal LSTM units, by a 
*tanh*-activated layer weighted by the input gate plus the forget gate 
influence:

$$ c_t = g_{forget} \odot c_{t-1} + g_{input} \odot \tanh(W_i \cdot [h_{t-1}, x_t] + b_i) $$ 

#### Hidden state (output):
The next hidden state is calculated by multiplying *tanh* of the cell state 
with the output gate:

$$ h_t = \tanh(c_t) \odot g_o $$

The following implementation is not optimised for performance but it 
follows strictly these formulas for clearness:

In [11]:
mutable struct Peep <: RecurrentUnit
    w; w_r; b            # input
    w_i; w_ir; c_i; b_i  # input gate
    w_o; w_or; c_o; b_o  # output gate
    w_f; w_fr; c_f; b_f  # forget gate
    c                    # cell state
    h                    # last hidden state

    function Peep(i, n; o...)   # i: fan-in, n: num cells
        w = param(n, i);    w_r = param(n, n); b = param0(n)
        w_i = param(n, i); w_ir = param(n, n); c_i = param0(n); b_i = param0(n)
        w_o = param(n, i); w_or = param(n, n); c_o = param0(n); b_o = param0(n)
        w_f = param(n, i); w_fr = param(n, n); c_f = param0(n); b_f = param(n, init=ones)
        c = init0(n)
        h = init0(n)

        new(w, w_r, b, 
             w_i, w_ir, c_i, b_i, 
             w_o, w_or, c_o, b_o, 
             w_f, w_fr, c_f, b_f, 
             c, h)
    end
end

In [12]:
function (l::Peep)(x)
    
    # gates:
    #
    i_gate = sigm.(l.w_i * x .+ l.w_ir * l.h .+ l.c_i .* l.c .* l.b_i)
    o_gate = sigm.(l.w_o * x .+ l.w_or * l.h .+ l.c_o .* l.c .* l.b_o)
    f_gate = sigm.(l.w_f * x .+ l.w_fr * l.h .+ l.c_f .* l.c .* l.b_f)
    
    # cell state:
    #
    c_temp = tanh.(l.w * x .+ l.w_r * l.h .+ l.b)     
    l.c = c_temp .* i_gate .+ l.c .* f_gate
    
    # hidden state (output):
    #
    l.h = tanh.(l.c) .* o_gate
    return l.h
end

## The model:

The ECG annotator model is defined as a RNN sequence tagger of one bidirectional layer
of the self-defined peephole LSTM units and a fully connected layer to predict
the state for each time step (beat or no beat).

The square loss is modified to increase the weight of false negative predictions,
because the training data includes much more non-beats as beats.

A thin network of only 8 peephole-LSTM units might be already able to solve the problem:

In [13]:
mutable struct BeatFinder <: DNN
    rlayer    # recurrent layer
    predict   # dense layer to generate 1/0 prediction
    n_units
    
    function BeatFinder(inputs, n_units)
        return new(Recurrent(inputs, n_units, u_type=Peep, bidirectional=true),
                   Linear(2*n_units, 1),
                   n_units)
    end
end

In [14]:
function (bf::BeatFinder)(x)
    x = bf.rlayer(x, return_all=true, h=0, c=0)
    x = bf.predict(x)
    x = reshape(x, size(x, 2),:)
    return x
end

function (bf::BeatFinder)(x,y)
    p = bf(x)
    weights = y .+ 1
    loss = mean(abs2, (p .- y) .* weights) # increase weight of peaks!
    return loss
end

In [15]:
bf = BeatFinder(2, 8);

In [16]:
print_network(bf)

NNHelferlein neural network summary:
BeatFinder,                                                        769 params
Details:
 
    Bidirectional Recurrent layer, 2 → 8 of type Peep,             752 params
    Linear layer 16 → 1, with identity,                             17 params
 
Total number of layers: 2
Total number of parameters: 769


2

# Training

As we have almost half a million windows in the training set, only one epoch is trained
after splitting 10% of the data for validation.    
Of course, LSTMs are slow - one epoch needs about 15 minutes (accuracy calculation for the 
full dataset takes almost the same time again):

In [17]:
bf = tb_train!(bf, Adam, dtrn, epochs=1, split=0.9, eval_freq=2, 
               acc_fun=peak_finder_acc, tb_name="$NAME");

Splitting dataset for training (90%) and validation (10%).
Training 1 epochs with 650 minibatches/epoch and 72 validation mbs.
Evaluation is performed every 325 minibatches with 15 mbs.
Watch the progress with TensorBoard at:
/data/aNN/Helferlein/logs/Beat-finder-8/2022-03-04T09-51-20


Progress: 100%|█████████████████████████████████████████| Time: 0:12:38


Training finished with:
Training loss:       0.014795012999899113
Training accuracy:   0.9870776848011605
Validation loss:     0.014813743389418555
Validation accuracy: 0.9869685808468984


# Validation

The quite narrow rnn reliably detects heartbeats in ECGs of reasonably good quality with F1-Score 
between 98% and 99% for training and validation data.
Errors only occur when the series is disturbed or at the very edge of windows.

Visual validation is possible with the `plot_test()` function, which 
plots a window of an ECG dataset together with the true heart beats (in *red*) and
the predictions (in *green*).

In [18]:
function window_split(df; win=200)
    x = []
    y = []
    i = 1
    while i <= nrow(df) - win+1
        wrange = i:i+win-1
        push!(x, permutedims(Array(df[wrange,[:cha1, :cha2]]), (2,1)))
        push!(y, Array(df[wrange,:beat]))
        i += win
    end
        
    return (x, y)
end

window_split (generic function with 1 method)

In [19]:
function plot_test(ecg; sec=0.0, steps=2000, win=200)
    i_start = Int(ceil(sec * 128)) + 1
    i_end = i_start + 129*200
    x,y = window_split(ecg[i_start:i_end,:], win=win)
    dtst = sequence_minibatch(x,y, 128, seq2seq=false, shuffle=false, partial=false)
    x,y = first(dtst)
    p = reshape(bf(x), :,1)[1:steps]

    df = ecg[i_start:i_start+steps-1,:]
    plotWindow(df.time, df.cha1, df.cha2, df.beat, p)
    peak_finder_acc(p, df.beat, verbose=3)
end

plot_test (generic function with 1 method)

In [ ]:
plot_test(nsr[16], sec=20000)

<img src="assets/62-vali-16.png">

In [ ]:
plot_test(nsr[18], sec=10)

<img src="assets/62-vali-18.png">